## Load MNIST data as numpy array

In [1]:
import h5py
import numpy as np

train_MNIST_dir = "MNIST_h5/training_data.h5" #MNIST training images in hdf5 format relative file path
test_MNIST_dir = "MNIST_h5/testing_data.h5"#MNIST testing images in hdf5 format relative file path

In [2]:
def read_h5(filepath):
    '''Reads MNIST training images and labels from the hdf5 file.
       Parameter
       ---------
       filepath : Path to the .h5 file | string
    '''
    file   = h5py.File(filepath, "r+") #open the hdf5 file
    images = np.array(file["/images"]).astype("uint8") #read the images dataset
    labels = np.array(file["/meta"]).astype("uint8")   #read the labels dataset (stored as meta)
    
    return (images,labels)

In [3]:
images, labels = read_h5(train_MNIST_dir)
test_images, test_labels = read_h5(test_MNIST_dir)

In [4]:
def one_hot_encoder(label_arr):
    '''Returns the given MNIST labels from np arrays of integers to np array of one hot labels.
       Parameter
       ---------
       label_arr : np array of MNIST integer labels
    '''
    total_labels  = label_arr.shape[0] #get the total number of labels
    one_hot_label = np.zeros([total_labels, 10]) #10 for num of classes in MNIST
    
    for i in range(label_arr.shape[0]): #loop through all the labels
        
        one_hot_label[i][int(label_arr[i])] = 1.0 #the label value will be marked as 1.0 at that specific index
        
    return one_hot_label #returns the np one-hot label 

In [ ]:
labels = one_hot_encoder(labels) #fetch the one-hot encoded labels
images = images.reshape(images.shape[0], 28,28,1) #images in numpy : [60000, 28, 28, 1]

test_labels = one_hot_encoder(test_labels)
test_images = test_images.reshape(test_images.shape[0], 28,28,1)

## Capsule Network

**Parameters**

In [ ]:
import tensorflow as tf
from tqdm import tqdm

batch_size           = 50
learning_rate        = 1e-4
epsilon              = 1e-9
epoch                = 20
height,width         = 28, 28
num_labels           = 10
primary_caps_vlength = 8
digit_caps_vlength   = 16
routing_iteration    = 3
m_plus               = 0.9
m_minus              = 0.1
lambda_              = 0.5
reg_scale            = 0.392
reconstruction       = False

**Capsule Network**

In [ ]:
def squash(capsule):
    '''Note that the input is a [batch_size, 1152, 1, 8, 1] tensor. 
       I.e. there are 1152 8-d vectors in each batch.
       To squash the vectors, we specificy the dimension the vector is in. In this case, axis is -2.
    '''
    #The output vector is in dimension -2 
    dot_product = tf.reduce_sum(tf.square(capsule), axis=-2, keepdims=True) 
    scalar_factor = dot_product/(1 + dot_product)/tf.sqrt(dot_product + epsilon)
    vec_squashed = scalar_factor * capsule
    return vec_squashed

In [ ]:
def routing(capsule_layer, num_capsules):
    
    #define a weight variable for one capsule first
    W = tf.get_variable('Weight', shape=(1, num_capsules, num_labels, primary_caps_vlength, digit_caps_vlength))
    b = tf.get_variable('Bias', shape=(1,1,num_labels, digit_caps_vlength,1 )) 
    W = tf.tile(W, [tf.shape(capsule_layer)[0], 1, 1 ,1 ,1]) #tiling just makes a copy of the same weight variable for all the items in the batch. It is still the same weight.
    x = tf.tile(capsule_layer, [1, 1, num_labels, 1, 1]) #this is set up for dynamic routing later
    u_hat = tf.matmul(W,x, transpose_a=True) #[batch_size, 1152, 10, 16, 1]
    u_hat_stopped = tf.stop_gradient(u_hat, name='stopped_gradient')
    
    #coefficients for dynamic routing
    #MUST be initialized to zero for every round
    b_ij = tf.zeros([tf.shape(capsule_layer)[0], num_capsules, num_labels, 1, 1], dtype=tf.float32)
    
    
    for r_iter in range(routing_iteration):
        
        #softmax so that the value of the coefficient is between 0 and 1
        #lower level capsules that contribute the most to a particular higher level capsule
        #will have the largest value
        c_ij = tf.nn.softmax(b_ij, axis=2)
        
        #last iteration
        if r_iter == routing_iteration - 1:
            
            s_j = tf.multiply(c_ij, u_hat) #multiply the coefficients with each capsule
            
            #reducing the sum at axis 1 makes the capsules with highest coefficient to 
            #contribute more and the lowest coefficient capsules to contirbute less
            s_j = tf.reduce_sum(s_j, axis=1, keepdims=True) + b
            
            v_j = squash(s_j)
            
        else:
            
            s_j = tf.multiply(c_ij, u_hat_stopped)
            
            #reducing the sum at axis 1 makes the capsules with highest coefficient to contribute more and the 
            #lowest coefficient capsules to contirbute less
            s_j = tf.reduce_sum(s_j, axis=1, keepdims=True) + b 
            v_j = squash(s_j)
            
            #make a copy at the number of capsules axis in
            #order to find the scalar product
            v_j_tiled = tf.tile(v_j, [1, num_capsules, 1, 1, 1]) 
            
            product = u_hat_stopped * v_j_tiled #[batch_size, 1152, 10, 16, 1]
            #by reducing the sum at axis 3, where the previous product produced new vectors, gives a scalar value.
            #Whichever capsules that agrees with each other will produce high valued vectors. Sum reduce would
            #add them all up together to bring a scalar value which then used for the softmax to enable the routing
            u_produce_v = tf.reduce_sum(product, axis=3, keepdims=True) #
            
            #append the contribution of each lower level capsules to the higher level capsules
            b_ij += u_produce_v
    
    return v_j

In [ ]:
X = tf.placeholder(tf.float32, shape=(None, height, width, 1))
Y = tf.placeholder(tf.float32, shape=(None, num_labels))

conv1 = tf.contrib.layers.conv2d(X, num_outputs=256, kernel_size=9, stride=1, padding='VALID', 
                                 activation_fn=tf.nn.relu) #output conv1 : [batch_size, 20, 20, 1]
conv2 = tf.contrib.layers.conv2d(conv1, num_outputs=256, kernel_size=9, stride=2, padding='VALID',
                                 activation_fn=tf.nn.relu) #output conv2 : [batch_size, 6, 6, 1]

#reshape the output of conv2 to [batch_size, 1152, 8, 1]
capsules = tf.reshape(conv2, (tf.shape(conv2)[0], -1, primary_caps_vlength, 1))
num_capsules = 6*6*32 #= 1152
primary_caps = squash(capsules) #non-linearity

#each primary capsule is multiplied by a weight matrix.The weight matrix will change the 8-D vectors
#to 16-D vectors. Furthermore, the number of capsules should also be reduced to 10.We would do that by dynamic 
#routing.
#However, before that, we need to create an extra rank after the num of capsules and tile it (2nd index
#starting from index 0) to 10. With that, the 1024 capsules can be reduced to 1 and remove that dimension. 
#The process of reducing the capsules to 10 is done by dynamic routing.
primary_caps = tf.reshape(primary_caps, shape=(tf.shape(capsules)[0], -1, 1, 8, 1 )) #create the extra rank 
#primary_caps.shape = [batch_size, 1152, 1, 8, 1]

digits = routing(primary_caps, num_capsules) # [batch_size, 1, 10, 16, 1]
digits = tf.squeeze(digits, axis=1) # [batch_size, 10, 16, 1]

#the length of each vectors in the digit capsule layer
v_lengths = tf.sqrt(tf.reduce_sum(tf.square(digits), axis=2, keepdims=True) + epsilon) #[batch_size, 10, 1, 1]

#Objective function for classification
max_l = tf.square(tf.maximum(0., m_plus - v_lengths))
max_r = tf.square(tf.maximum(0., v_lengths - m_minus))

max_l = tf.reshape(max_l, shape=(tf.shape(digits)[0], -1))
max_r = tf.reshape(max_r, shape=(tf.shape(digits)[0], -1))
T_c = Y

L_c = T_c * max_l + lambda_*(1-T_c)*max_r
margin_loss = tf.reduce_mean(tf.reduce_sum(L_c, axis=1)) #test without reduce mean later


#RECONSTRUCTION
#First step is to mask out all the vectors from digit capsules except the correct class vector.
#By performing an element-wise multiplication between the 10 capsules and the Y vector
mask = tf.multiply(tf.squeeze(digits), tf.reshape(Y, (-1, num_labels, 1)))
vector_j = tf.reshape(mask, shape=(tf.shape(digits)[0], 160)) #reshape into [batch_size, 160]
fc1 = tf.contrib.layers.fully_connected(vector_j, num_outputs=512, activation_fn=tf.nn.relu)
fc2 = tf.contrib.layers.fully_connected(fc1, num_outputs=1024, activation_fn=tf.nn.relu)
decoded = tf.contrib.layers.fully_connected(fc2, num_outputs=height*width*1, activation_fn=tf.sigmoid)

#RECONSTUCTION LOSS
origin   = tf.reshape(X, shape=(tf.shape(digits)[0], -1))
squared  = tf.square(decoded - origin)
reconst_err = tf.reduce_mean(squared)

total_loss = margin_loss + reg_scale + reconst_err #total for both classification and reconstruction

optimizer_classification = tf.train.AdamOptimizer(learning_rate).minimize(margin_loss) #for classification
optimizer_decoder = tf.train.AdamOptimizer(learning_rate).minimize(total_loss) #for reconstruction

softmax_v = tf.nn.softmax(v_lengths, axis=1)
argmax_idx = tf.argmax(softmax_v, axis=1)
argmax_idx = tf.reshape(argmax_idx, shape=(tf.shape(conv2)[0],))
correct_prediction = tf.equal(argmax_idx, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float32'))

**Training & Testing Session**

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

total_images = images.shape[0] #total number of training images
total_test_images = test_images.shape[0] #total number of testing images

for epoch_iter in range(epoch):
    counter = 0
    loss = 0
    accuracy_total = 0
    test_counter = 0
    test_accuracy_total = 0
    test_loss = 0
    
    #FOR TRAINING 
    for index in tqdm(range(0, total_images, batch_size)):
        
        end_batch = index + batch_size 
        
        if end_batch >= total_images : end_batch = None #to prevent the last index to go beyond the data size
        
        if reconstruction:
            #optimize for classification and reconstruction
            loss_, accuracy_,_ = sess.run([total_loss,accuracy, optimizer_decoder], 
                            feed_dict={X:images[index:end_batch], Y:labels[index:end_batch]})
            
            loss += loss_
            accuracy_total += accuracy_
            
            counter += 1
            
        else:
            #optimize only for classification
            
            loss_, accuracy_,_ = sess.run([margin_loss, accuracy, optimizer_classification], 
                            feed_dict={X:images[index:end_batch], Y:labels[index:end_batch]})
            
            loss += loss_
            accuracy_total += accuracy_
            
            counter += 1
            
    #FOR TESTING
    for test_index in tqdm(range(0, total_test_images, batch_size)):
        
        end_batch_test = test_index + batch_size 
        
        if end_batch_test >= total_test_images : end_batch_test = None #to prevent the last index to go beyond the data size
        
        if reconstruction:
           
            test_loss_, test_accuracy_ = sess.run([total_loss,accuracy], 
                            feed_dict={X:test_images[test_index:end_batch_test], Y:test_labels[test_index:end_batch_test]})
            
            test_loss += test_loss_
            test_accuracy_total += test_accuracy_
            
            test_counter += 1
            
        else:
          
            
            test_loss_, test_accuracy_= sess.run([margin_loss, accuracy], 
                            feed_dict={X:test_images[test_index:end_batch_test], Y:test_labels[test_index:end_batch_test]})
            
            test_loss += loss_
            test_accuracy_total += test_accuracy_
            
            test_counter += 1
        
    
    print("Epoch %d, training loss %g, training accuracy %g"%(epoch_iter, loss, accuracy_total/counter))
    print("Epoch %d, test loss     %g, test accuracy     %g"%(epoch_iter, test_loss, test_accuracy_total/test_counter))

sess.close()

  0%|          | 0/1200 [00:00<?, ?it/s]

Epoch 0, training loss 52.7136, training accuracy 0.966033
Epoch 0, test loss     1.12137, test accuracy     0.9871


  0%|          | 0/1200 [00:00<?, ?it/s]

Epoch 1, training loss 14.8451, training accuracy 0.99075
Epoch 1, test loss     0.345918, test accuracy     0.9904


  0%|          | 0/1200 [00:00<?, ?it/s]

...

  0%|          | 0/1200 [00:00<?, ?it/s]

Epoch 9, training loss 1.064, training accuracy 0.999717
Epoch 9, test loss     0.00112724, test accuracy     0.9934


  0%|          | 0/1200 [00:00<?, ?it/s]

Epoch 10, training loss 0.933205, training accuracy 0.9998
Epoch 10, test loss     0.00343827, test accuracy     0.9943


 14%|█▎        | 164/1200 [01:08<07:29,  2.31it/s]